In [1]:
!pip install -q transformers datasets accelerate

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset as HFDataset

train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

def combine_text(row):
    return f"[PROMPT] {row['prompt']} [A] {row['response_a']} [B] {row['response_b']}"

train["text"] = train.apply(combine_text, axis=1)
test["text"] = test.apply(combine_text, axis=1)

label_map = {"model_a": 0, "model_b": 1, "tie": 2}
train["label"] = train["winner"].map(label_map)

model_name = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

train_ds = HFDataset.from_pandas(train[["text", "label"]]).map(tokenize_function, batched=True)
test_ds = HFDataset.from_pandas(test[["text"]]).map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
)

trainer.train()

preds = trainer.predict(test_ds).predictions
probs = torch.nn.functional.softmax(torch.tensor(preds), dim=-1).numpy()

submission = pd.DataFrame({
    "id": test["id"],
    "winner_model_a": probs[:, 0],
    "winner_model_b": probs[:, 1],
    "winner_tie": probs[:, 2]
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission saved!")
